# Core Imports

In [ ]:
# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import IPythonConsole

DIM    = 300
ASPECT = 3/2
IPythonConsole.molSize = (int(ASPECT*DIM), DIM)   # Change image size

# Static Paths
RAW_DATA_DIR  = Path('raw_monomer_data')
PROC_DATA_DIR = Path('processed_monomer_data')
RXN_FILES_DIR = Path('poly_rxns')
# RXN_FILES_DIR = Path('rxn_smarts')

# Inspecting and filtering expanded dataset

In [ ]:
from polymerist.monomers import specification

input_data_path = RAW_DATA_DIR / '20231114_polyid_data_density_DP2-6 - 1,2 monomers.csv'
df = pd.read_csv(input_data_path)

## Filter by number of monomer fragments in monomer SMILES

### Verify that the num_monomers fields is accurate

In [ ]:
num_mols_in_smiles = lambda smiles : len(
    Chem.GetMolFrags(
        Chem.MolFromSmiles(smiles)
    )
)

reported_num_monomers = df['num_monomers']
actual_num_monomers   = df.apply(lambda row : num_mols_in_smiles(row['smiles_monomer']), axis=1)

assert( (reported_num_monomers == actual_num_monomers).all() )

In [ ]:
df = df[df['num_monomers'] == 2]
display(df)

## Filter by chemistry

### Insert column of expanded monomer SMILES for combined mols

In [ ]:
col_labels = df.columns.to_list()
# col_labels.insert(1, 'smiles_expanded')
# df = df.reindex(columns=col_labels) # this need to be done BEFORE actually inserting the new column to avoid duplication
df.insert(
    loc=1,
    column='smiles_expanded',
    value=df['smiles_monomer'].apply(lambda smiles : specification.expanded_SMILES(smiles, assign_map_nums=False))
)

### Remove entries which contain any undesired atoms

In [ ]:
from rdkit.Chem import rdqueries
from polymerist.rdutils.smileslib import queries

atom_queries = {
    'silicon' : Chem.MolFromSmarts('[Si]'),
    'sulfur'  : Chem.MolFromSmarts('[S]'),
    'metal'   : queries.SPECIAL_QUERY_MOLS['metal'],
    # 'halogen' : queries.SPECIAL_QUERY_MOLS['halogen'],
}

In [ ]:
undes_atoms_present = df.apply(lambda row : 
    queries.matching_dict_from_substruct_dict(Chem.MolFromSmiles(row['smiles_expanded'], sanitize=False), atom_queries).values(),
    axis=1,
    result_type='expand'
)

df[[f'{atom_chem}_present' for atom_chem in atom_queries.keys()]] = undes_atoms_present

In [ ]:
df = df[~undes_atoms_present.any(axis=1)]

## Filter by reactive functional groups, placing fragments in correct order

### Load pre-defined functional group SMARTS queries

In [ ]:
with Path('poly_rxns/fn_group_smarts.json').open('r') as file:
    fn_group_smarts = json.load(file)

fn_groups = {
    group_name : Chem.MolFromSmarts(smarts)
        for group_name, smarts in fn_group_smarts.items()
}

### Backmap reactions and their functional groups

In [ ]:
from polymerist.rdutils.reactions import reactions

rxn_backmap = { # map NREL dataset mechanism names to pre-made rxn template names
    'amide'     : 'polyamide',
    'carbonate' : 'polycarbonate_phosgene',
    'ester'     : 'polyester',
    'imide'     : 'polyimide',
    'urethane'  : 'polyurethane_isocyanate',
    'NIPU'      : 'polyurethane_nonisocyanate',
    'vinyl'     : 'polyvinyl_head_tail'
}

assert(set(df['mechanism'].unique()).issubset(set(rxn_backmap.keys()))) # verify that we've mapped all reactions

backmap_path = RXN_FILES_DIR / 'rxn_backmap.json'
with backmap_path.open('w') as backmap_file:
    json.dump(rxn_backmap, backmap_file, indent=4)

In [ ]:
rxns = {
    rxnname : reactions.AnnotatedReaction.from_rxnfile(RXN_FILES_DIR / f'{rxnname}.rxn')
        for rxnname in rxn_backmap.values()
}

with (RXN_FILES_DIR / 'rxn_groups.json').open('r') as file: # load table of functional group for each reaction
    rxn_groups = json.load(file)

In [ ]:
df.insert( # insert column applying backmap to listed mechanism name
    loc=df.columns.to_list().index('mechanism') + 1, # insert after existing mechanism column
    column='rxn_name',
    value=df.mechanism.map(rxn_backmap)
)

### Identify if fragments match a defined rxn, and if so, in what order they should appear

In [ ]:
from polymerist.maths.combinatorics.sequences import bin_ids_forming_sequence

def get_ordered_monomer_fragments(row : pd.Series) -> list[Optional[str], Optional[str]]:
    '''Returns expanded SMILES each individual monomer in the correct order for their respective reaction
    or all NoneType if no such match can be found'''
    targ_groups = rxn_groups[rxn_backmap[row.mechanism]]
    frag_smiles = row['smiles_expanded'].split('.') # easier in practice than generating fragments of combined mol, turning fragments to SMILES, and expanding those SMILES
    choice_bins = [
        queries.matching_labels_from_substruct_dict(Chem.MolFromSmiles(smiles, sanitize=False), fn_groups)
            for smiles in frag_smiles
    ]

    nfrags = len(frag_smiles) # avoids relying on prior data sanitization
    # nfrags = row['num_monomers']
    all_indices = set(range(nfrags))

    for ids in bin_ids_forming_sequence(sequence=targ_groups, choice_bins=choice_bins):
        if set(ids) == all_indices:
            return [frag_smiles[i] for i in ids] # return fragments in the order defined by the sequence
    else:
        return [None for _ in all_indices] # return all None if no covering order is found

### Insert individual monomers into table, remove rows with no monomers

In [ ]:
reacting_monomers = df.apply(get_ordered_monomer_fragments, axis=1, result_type='expand')
for (i, col) in reacting_monomers.items():
    df.insert(
        loc=i+3,
        column=f'smiles_expanded_monomer_{i+1}',
        value=col
    )

df = df[reacting_monomers.notnull().all(axis=1)]

## Filtering by IUPAC name query to chemical databases

In [ ]:
import cirpy
import pubchempy as pcp
import chemspipy as csp

def get_IUPAC_name(smiles : str) -> Optional[str]:
    '''Attempts to fetch the IUPAC name for the molecule described by a SMILES string from online chemical resolution services
    Returns the fetched IUPAC name, or NoneType if both queries are unsuccessful'''
    try:
        pcquery = pcp.get_compounds(smiles, namespace='smiles')
        pccomp = pcquery.pop()
        return getattr(pccomp, 'iupac_name')
    except Exception as e:
        print(type(e), e)
        return None

In [ ]:
IUPAC_names = df[['smiles_expanded_monomer_1', 'smiles_expanded_monomer_2']].map(get_IUPAC_name) # this takes a while due to the HTTP query
IUPAC_names

In [ ]:
IUPAC_names.columns = (0, 1)
for (i, col) in IUPAC_names.items():
    df.insert(
        loc=i+5,
        column=f'IUPAC_monomer_{i+1}',
        value=col
    )

df = df[IUPAC_names.notnull().all(axis=1)]

## Save filtered DataFrame for next steps to avoid reprocessing

In [ ]:
out_data_path = PROC_DATA_DIR / '20231114_polyid_data_density_FILTERED.csv'
df.to_csv(out_data_path)